# Aroma Notebook for Colab

## Config

In [ ]:
#@title Configurations

#@markdown Aroma Repository
repo_url = 'https://github.com/lumiknit/aroma' #@param {type:"string"}

#@markdown Huggingface model repo & comma-separated ignore list
hf_model_repo = 'lumiknit/diffusers-model-archive' #@param {type:"string"}

#@markdown WebUI Basic Auth Info
#@markdown Note that the length of password **MUST BE** at least 8.
webui_username = '' #@param {type:"string"}
webui_password = '' #@param {type:"string"}

#@markdown Daemon password
daemon_password = '' #@param {type:"string"}

#@markdown **NOTE** To get ngrok token, visit https://dashboard.ngrok.com/get-started/your-authtoken for ngrok token
ngrok_token = '' #@param {type:"string"}

## Setup

In [ ]:
!rm -rf /ws

In [ ]:
!curl -sL https://deb.nodesource.com/setup_18.x | sudo -E bash -
!apt-get install -y jq nodejs
!pip install pyngrok pytorch-lightning numpy pillow scipy ftfy transformers diffusers accelerate safetensors omegaconf huggingface_hub tensorrt

In [ ]:
# Clone repo and setup
%cd /
!git clone $repo_url /ws/aroma
%cd /ws/aroma
!git lfs install
!mkdir -p /ws/aroma/models /ws/aroma/archives /ws/aroma/outputs /ws/aroma/state

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id=hf_model_repo,
    allow_patterns=["textual_inversions/*", "anything-v4.5/*"],
    local_dir="/ws/aroma/models")

In [ ]:
# Change host and port to 0.0.0.0:9125
import json

config = {
    'webui': {
        'host': '0.0.0.0',
        'port': 9125,
        'username': webui_username,
        'password': webui_password,
    },
    'save_raw': False,
    'password': daemon_password,
    'init_values': {
        'model': {
            'path': "anything-v4.5"
        },
        'textual_inversions': [
            #"textual_inversions/EasyNegative.safetensors",
            "textual_inversions/EasyNegativeV2.safetensors",
            "textual_inversions/bad-picture-chill-75v.pt",
            "textual_inversions/ng_deepnegative_v1_75t.pt",
            "textual_inversions/realisticvision-negative-embedding.pt",
            "textual_inversions/verybadimagenegative_v1.3.pt",
        ],
        'params': {
            "prompt": "cat",
            "negative_prompt": "EasyNegative, ng_deepnegative_v1_75t, veryBadImageNegative_v1.3, bad-picture-chill-75v, bad quality, low quality, bad anatomy, nsfw",
            'width': 512,
            'height': 512,
            "highres_fix": [
                {
                  "width": 768,
                  "height": 768,
                  "strength": 0.5
                }
            ]
        }
    }
}
config_json = json.dumps(config)

!echo '{config_json}' > /ws/aroma/config.json
!touch /ws/aroma/state/values.json

In [ ]:
# Install node packages
!cd /ws/aroma/ui && npm install -g npm && npm install
!node --version

# Run

In [ ]:
import os
import threading
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9125, auth=f"{webui_username}:{webui_password}").public_url
print(f"{public_url}")
!node ui/main.js